Prepare enviroment

In [ ]:
! git clone https://github.com/moses-smt/mosesdecoder.git
! git clone https://github.com/rsennrich/subword-nmt.git
! git clone https://github.com/pytorch/fairseq
! pip install sacremoses
! pip install wandb
! pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
%cd fairseq
!pip install fairseq==0.12.2
%cd ..
! cp -r /content/subword-nmt /content/fairseq/examples/backtranslation
! cp -r /content/mosesdecoder /content/fairseq/examples/backtranslation

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148097, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 148097 (delta 323), reused 441 (delta 292), pack-reused 147572
Receiving objects: 100% (148097/148097), 129.88 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (114349/114349), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 597 (delta 8), reused 12 (delta 4), pack-reused 576
Receiving objects: 100% (597/597), 252.23 KiB | 1.36 MiB/s, done.
Resolving deltas: 100% (357/357), done.
Cloning into 'fairseq'...
remote: Enumerating objects: 34294, done.
remote: Total 34294 (delta 0), reused 0 (delta 0), pack-reused 34294
Receiving objects: 100% (34294/34294), 23.75 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (24982/24982), done.
Looking in indexes: https://pypi.org/simple

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! mkdir /content/drive/MyDrive/NLP

Mounted at /content/drive


In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Download Europarl corpus for base model

In [ ]:
! mkdir Europarl
%cd Europarl
! wget https://object.pouta.csc.fi/OPUS-Europarl/v7/moses/de-en.txt.zip
! unzip  de-en.txt.zip
%cd ..

/content/Europarl
--2023-01-14 15:07:18--  https://object.pouta.csc.fi/OPUS-Europarl/v8/moses/de-en.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211805161 (202M) [application/zip]
Saving to: ‘de-en.txt.zip’

de-en.txt.zip       100%[===================>] 201.99M  12.8MB/s    in 18s     

2023-01-14 15:07:37 (11.4 MB/s) - ‘de-en.txt.zip’ saved [211805161/211805161]

Archive:  de-en.txt.zip
  inflating: README                  
  inflating: LICENSE                 
  inflating: Europarl.de-en.de       
  inflating: Europarl.de-en.en       
  inflating: Europarl.de-en.xml      
/content


In [ ]:
! cp -R Europarl drive/MyDrive/NLP/

In [ ]:
! cp -R drive/MyDrive/NLP/Europarl Europarl

Text preprocessing

---



In [ ]:
%%shell
#!/bin/bash

SRC_FOLDER=Europarl
lang=de-en
src=en
tgt=de

for l in $src $tgt; do
    awk 'NR%2== 1  {print $0}' $SRC_FOLDER/Europarl.$lang.$l > $SRC_FOLDER/Europarl2.$lang.$l
done
for l in $src $tgt; do
    rm $SRC_FOLDER/Europarl.$lang.$l
    mv $SRC_FOLDER/Europarl2.$lang.$l $SRC_FOLDER/Europarl.$lang.$l
done

In [ ]:
%%shell
#!/bin/bash
SRC_FOLDER=Europarl
TRG_FOLDER=EuroparlPrepered
SCRIPTS=mosesdecoder/scripts
TOKENIZER=$SCRIPTS/tokenizer/tokenizer.perl
CLEAN=$SCRIPTS/training/clean-corpus-n.perl
NORM_PUNC=$SCRIPTS/tokenizer/normalize-punctuation.perl
REM_NON_PRINT_CHAR=$SCRIPTS/tokenizer/remove-non-printing-char.perl
BPEROOT=subword-nmt/subword_nmt
BPE_TOKENS=32000

lang=de-en
src=en
tgt=de

echo "splitting train, test and valid..."
mkdir $TRG_FOLDER
mkdir $TRG_FOLDER/CODE
for l in $src $tgt; do
    awk '{if(NR%450<448) print $0;}' $SRC_FOLDER/Europarl.$lang.$l > $TRG_FOLDER/train.$l
    awk 'NR%500== 448  {print $0}' $SRC_FOLDER/Europarl.$lang.$l > $TRG_FOLDER/test.$l
    awk 'NR%500== 449  {print $0}' $SRC_FOLDER/Europarl.$lang.$l > $TRG_FOLDER/valid.$l
done

echo "preoprocessing train data..."
for l in $src $tgt; do
    cat $TRG_FOLDER/train.$l | \
      perl $NORM_PUNC $l | \
      perl $REM_NON_PRINT_CHAR | \
      perl $TOKENIZER -threads 8 -a -l $l >> $TRG_FOLDER/trainPREP.$l
done

echo "preoprocessing valid data..."
for l in $src $tgt; do
    cat $TRG_FOLDER/valid.$l | \
      perl $NORM_PUNC $l | \
      perl $REM_NON_PRINT_CHAR | \
      perl $TOKENIZER -threads 8 -a -l $l >> $TRG_FOLDER/validPREP.$l
done

echo "preoprocessing test data..."
for l in $src $tgt; do
    cat $TRG_FOLDER/test.$l | \
      perl $NORM_PUNC $l | \
      perl $REM_NON_PRINT_CHAR | \
      perl $TOKENIZER -threads 8 -a -l $l >> $TRG_FOLDER/testPREP.$l
done

TRAIN=trainPREP.de-en
BPE_CODE=code
for l in $src $tgt; do
    cat $TRG_FOLDER/trainPREP.$l >> $TRG_FOLDER/$TRAIN
done

echo "learn_bpe.py on ${TRAIN}..."
python $BPEROOT/learn_bpe.py -s $BPE_TOKENS < $TRG_FOLDER/$TRAIN > $TRG_FOLDER/CODE/$BPE_CODE

for L in $src $tgt; do
    for f in trainPREP.$L validPREP.$L testPREP.$L; do
        echo "apply_bpe.py to ${f}..."
        python $BPEROOT/apply_bpe.py -c $TRG_FOLDER/CODE/$BPE_CODE < $TRG_FOLDER/$f > $TRG_FOLDER/CODE/bpe.$f
    done
done
mkdir $TRG_FOLDER/PREPERED
perl $CLEAN -ratio 1.5 $TRG_FOLDER/CODE/bpe.trainPREP $src $tgt $TRG_FOLDER/PREPERED/train 2 250
perl $CLEAN -ratio 1.5 $TRG_FOLDER/CODE/bpe.validPREP $src $tgt $TRG_FOLDER/PREPERED/valid 2 250
perl $CLEAN -ratio 1.5 $TRG_FOLDER/CODE/bpe.testPREP $src $tgt $TRG_FOLDER/PREPERED/test 2 250

cp $TRG_FOLDER/CODE/code $TRG_FOLDER/PREPERED/code
cp -R EuroparlPrepered drive/MyDrive/NLP/

splitting train, test and valid...
preoprocessing train data...
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: de
Number of threads: 8
preoprocessing valid data...
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: de
Number of threads: 8
preoprocessing test data...
Tokenizer Version 1.1
Language: en
Number of threads: 8
Tokenizer Version 1.1
Language: de
Number of threads: 8
learn_bpe.py on trainPREP.de-en...
100% 32000/32000 [01:31<00:00, 347.94it/s]
apply_bpe.py to trainPREP.en...
apply_bpe.py to validPREP.en...
apply_bpe.py to testPREP.en...
apply_bpe.py to trainPREP.de...
apply_bpe.py to validPREP.de...
apply_bpe.py to testPREP.de...
clean-corpus.perl: processing EuroparlPrepered/CODE/bpe.trainPREP.en & .de to EuroparlPrepered/PREPERED/train, cutoff 2-250, ratio 1.5
..........(100000)..........(200000)..........(300000)..........(400000)..........(500000)..........(600000)..........(700000)..........(800

In [ ]:
! cp -R drive/MyDrive/NLP/EuroparPrepered EuroparlPrepered

In [ ]:
! mkdir EuroparlTokenized.en-de
! fairseq-preprocess \
    --joined-dictionary \
    --source-lang en --target-lang de \
    --trainpref EuroparlPrepered/PREPERED/train --validpref EuroparlPrepered/PREPERED/valid --testpref EuroparlPrepered/PREPERED/test \
    --destdir EuroparlTokenized.en-de
! cp -R EuroparlTokenized.en-de drive/MyDrive/NLP/

2022-12-27 18:31:56 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-12-27 18:31:56 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='EuroparlTokenized.en-de', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma',

In [ ]:
! cp -R drive/MyDrive/NLP/EuroparlTokenized.en-de EuroparlTokenized.en-de

In [ ]:
! mkdir -p EuroparlModel
! mkdir -p EuroparlModel/transformer

In [ ]:
! CUDA_VISIBLE_DEVICES=0 fairseq-train --fp16 drive/MyDrive/NLP/EuroparlTokenized.en-de \
    --source-lang en --target-lang de \
    --arch transformer_wmt_en_de 6 \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 11000 \
    --save-dir drive/MyDrive/NLP/EuroparlModel/transformer \
    --wandb-project "Uczenie modelu podstawowego"

2023-01-06 10:29:00 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-06 10:29:01 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Uczenie modelu podstawowego', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': Fal

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/EuroparlModel/transformer
bash examples/backtranslation/sacrebleu.sh \
    wmt17 \
    en-de \
    ../drive/MyDrive/NLP/EuroparlTokenized.en-de \
    ../drive/MyDrive/NLP/EuroparlPrepered/CODE/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

2023-01-06 10:48:10 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-06 10:48:12 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/EuroparlModel/transformer
bash examples/backtranslation/sacrebleu.sh \
    wmt08/europarl \
    en-de \
    ../drive/MyDrive/NLP/EuroparlTokenized.en-de \
    ../drive/MyDrive/NLP/EuroparlPrepered/CODE/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

2023-01-06 10:46:48 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-06 10:46:50 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging



> BACKTRANSLATION



Prepare paraller corpora

In [ ]:
! cp drive/MyDrive/NLP/scripts/prepare-wmt18en2de.sh /content/fairseq/examples/backtranslation/

In [ ]:
%%shell
cd fairseq/examples/backtranslation/
bash prepare-wmt18en2de.sh

In [ ]:
! cp /content/fairseq/examples/backtranslation/prepare-wmt18en2de.sh drive/MyDrive/NLP/scripts/

In [ ]:
%%shell
cd ./fairseq/
TEXT=examples/backtranslation/wmt18_en_de
fairseq-preprocess \
    --joined-dictionary \
    --source-lang en --target-lang de \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/wmt18_en_de --thresholdtgt 0 --thresholdsrc 0 \
    --workers 20

In [ ]:
! cp ./fairseq/examples/backtranslation/wmt18_en_de/code ./fairseq/data-bin/wmt18_en_de/code

In [ ]:
! cp -R /content/fairseq/data-bin drive/MyDrive/NLP/data-bin

Train intermidiete model

In [ ]:
! cp -R drive/MyDrive/NLP/data-bin /content/fairseq/data-bin

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_de_en_parallel
fairseq-train --fp16 \
    data-bin/wmt18_en_de \
    --source-lang de --target-lang en \
    --arch transformer_wmt_en_de --share-all-embeddings \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.001 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 15000 \
    --save-dir $CHECKPOINT_DIR \
    --wandb-project "transformer_wmt_de_en"

evaluate model

In [ ]:
! cp drive/MyDrive/NLP/scripts/sacrebleu.sh /content/fairseq/examples/backtranslation/

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_de_en_parallel
bash examples/backtranslation/sacrebleu.sh \
    wmt17 \
    de-en \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_de_en_parallel
bash examples/backtranslation/sacrebleu.sh \
    wmt08/europarl \
    de-en \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

In [ ]:
! cp /content/fairseq/examples/backtranslation/sacrebleu.sh drive/MyDrive/NLP/scripts/

prepare monolingual data

In [ ]:
! cp drive/MyDrive/NLP/scripts/prepare-de-monolingual.sh /content/fairseq/examples/backtranslation/

In [ ]:
! cp -r /content/fairseq/data-bin/wmt18_en_de /content/fairseq/examples/backtranslation

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/examples/backtranslation/
bash prepare-de-monolingual.sh
cd ../..

In [ ]:
! cp /content/fairseq/examples/backtranslation/prepare-de-monolingual.sh drive/MyDrive/NLP/scripts/

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
TEXT=examples/backtranslation/wmt18_de_mono
for SHARD in $(seq -f "%02g" 0 24); do \
    fairseq-preprocess \
        --only-source \
        --source-lang de --target-lang en \
        --joined-dictionary \
        --srcdict data-bin/wmt18_en_de/dict.de.txt \
        --testpref $TEXT/bpe.monolingual.dedup.${SHARD} \
        --destdir data-bin/wmt18_de_mono/shard${SHARD} \
        --workers 20; \
    cp data-bin/wmt18_en_de/dict.en.txt data-bin/wmt18_de_mono/shard${SHARD}/; \
done

In [ ]:
! cp -R /content/fairseq/data-bin/wmt18_de_mono drive/MyDrive/NLP/data-bin/wmt18_de_mono

beam search

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
mkdir backtranslation_output
for SHARD in $(seq -f "%02g" 0 24); do \
    fairseq-generate --fp16 \
        data-bin/wmt18_de_mono/shard${SHARD} \
        --path ../drive/MyDrive/NLP/checkpoints_de_en_parallel/checkpoint_best.pt \
        --skip-invalid-size-inputs-valid-test \
        --max-tokens 4096 \
        --sampling --beam 5 \
        --nbest 5 \
    > backtranslation_output/sampling.shard${SHARD}.out; \
done

In [ ]:
! cp -R /content/fairseq/backtranslation_output drive/MyDrive/NLP/backtranslation_output_beam

top10

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
mkdir backtranslation_output
for SHARD in $(seq -f "%02g" 0 24); do \
    fairseq-generate --fp16 \
        data-bin/wmt18_de_mono/shard${SHARD} \
        --path ../drive/MyDrive/NLP/checkpoints_de_en_parallel/checkpoint_best.pt \
        --skip-invalid-size-inputs-valid-test \
        --max-tokens 4096 \
        --sampling --beam 1 \
        --sampling-topk 10 \
    > backtranslation_output/sampling.shard${SHARD}.out; \
done

In [ ]:
! cp -R /content/fairseq/backtranslation_output drive/MyDrive/NLP/backtranslation_output_top10

Combaining monolingual data with parallel data

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
python examples/backtranslation/extract_bt_data.py \
    --minlen 1 --maxlen 250 --ratio 1.5 \
    --output backtranslation_output/bt_data --srclang en --tgtlang de \
    backtranslation_output/sampling.shard*.out

In [ ]:
! wc -l fairseq/backtranslation_output/bt_data.{en,de}

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
TEXT=backtranslation_output
fairseq-preprocess \
    --source-lang en --target-lang de \
    --joined-dictionary \
    --srcdict data-bin/wmt18_en_de/dict.en.txt \
    --trainpref $TEXT/bt_data \
    --destdir data-bin/wmt18_en_de_bt_top10 \
    --workers 20

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
PARA_DATA=$(readlink -f data-bin/wmt18_en_de)
BT_DATA=$(readlink -f data-bin/wmt18_en_de_bt_top10)
COMB_DATA=data-bin/wmt18_en_de_para_plus_bt_top10
mkdir -p $COMB_DATA
for LANG in en de; do \
    ln -s ${PARA_DATA}/dict.$LANG.txt ${COMB_DATA}/dict.$LANG.txt; \
    for EXT in bin idx; do \
        ln -s ${PARA_DATA}/train.en-de.$LANG.$EXT ${COMB_DATA}/train.en-de.$LANG.$EXT; \
        ln -s ${BT_DATA}/train.en-de.$LANG.$EXT ${COMB_DATA}/train1.en-de.$LANG.$EXT; \
        ln -s ${PARA_DATA}/valid.en-de.$LANG.$EXT ${COMB_DATA}/valid.en-de.$LANG.$EXT; \
        ln -s ${PARA_DATA}/test.en-de.$LANG.$EXT ${COMB_DATA}/test.en-de.$LANG.$EXT; \
    done; \
done

In [ ]:
! cp -R /content/fairseq/data-bin/wmt18_en_de_para_plus_bt_top10 drive/MyDrive/NLP/wmt18_en_de_para_plus_bt_top10

In [ ]:
! cp -R /content/fairseq/data-bin/wmt18_en_de_bt_top10 drive/MyDrive/NLP/wmt18_en_de_bt_top10

Train an English-German model over the combined parallel + BT data

In [ ]:
! cp -R drive/MyDrive/NLP/data-bin /content/fairseq/data-bin

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_beam
fairseq-train --fp16 \
    data-bin/wmt18_en_de_para_plus_bt_beam \
    --upsample-primary 1 \
    --source-lang en --target-lang de \
    --arch transformer_wmt_en_de --share-all-embeddings \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0007 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 15000 \
    --save-dir $CHECKPOINT_DIR \
    --wandb-project "transformer_wmt_en_de_beam"

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_top10
fairseq-train --fp16 \
    data-bin/wmt18_en_de_para_plus_bt_top10 \
    --upsample-primary 1 \
    --source-lang en --target-lang de \
    --arch transformer_wmt_en_de --share-all-embeddings \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0007 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 15000 \
    --save-dir $CHECKPOINT_DIR \
    --wandb-project "transformer_wmt_en_de_top10"

Training with backtranslation

In [ ]:
! cp -R drive/MyDrive/NLP/data-bin /content/fairseq/data-bin

In [ ]:
! cp drive/MyDrive/NLP/sacrebleu.sh /content/fairseq/examples/backtranslation/

Beam

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_beam
fairseq-train --fp16 \
    data-bin/wmt18_en_de_para_plus_bt_beam \
    --upsample-primary 1 \
    --source-lang en --target-lang de \
    --arch transformer_wmt_en_de --share-all-embeddings \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0007 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 11000 \
    --save-dir $CHECKPOINT_DIR \
    --wandb-project "transformer_wmt_en_de_beam"

2023-01-05 17:42:31 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-05 17:42:33 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'transformer_wmt_en_de_beam', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': Fals

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_beam
bash examples/backtranslation/sacrebleu.sh \
    wmt17 \
    en-de \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

2023-01-05 20:13:08 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-05 20:13:09 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_beam
bash examples/backtranslation/sacrebleu.sh \
    wmt08/europarl \
    en-de \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

2023-01-05 20:01:20 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-05 20:01:21 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging

Top

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_top10
fairseq-train --fp16 \
    data-bin/wmt18_en_de_para_plus_bt_top10 \
    --upsample-primary 1 \
    --source-lang en --target-lang de \
    --arch transformer_wmt_en_de --share-all-embeddings \
    --dropout 0.3 --weight-decay 0.0 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 0.0007 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --max-tokens 3584 --update-freq 16 \
    --max-update 11000 \
    --save-dir $CHECKPOINT_DIR \
    --wandb-project "transformer_wmt_en_de_top10"

2023-01-11 07:51:15 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-11 07:51:17 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'transformer_wmt_en_de_top10', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': Fal

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_top10
bash examples/backtranslation/sacrebleu.sh \
    wmt17 \
    en-de \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

sacreBLEU: Downloading http://data.statmt.org/wmt17/translation-task/test.tgz to /root/.sacrebleu/wmt17/raw/wmt17.test.tgz
sacreBLEU: Extracting /root/.sacrebleu/wmt17/raw/wmt17.test.tgz to /root/.sacrebleu/wmt17/raw
2023-01-11 11:03:19 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-11 11:03:21 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries':

In [ ]:
%%shell
#!/bin/bash
cd ./fairseq/
CHECKPOINT_DIR=../drive/MyDrive/NLP/checkpoints_en_de_parallel_bt_top10
bash examples/backtranslation/sacrebleu.sh \
    wmt08/europarl \
    en-de \
    data-bin/wmt18_en_de \
    data-bin/wmt18_en_de/code \
    $CHECKPOINT_DIR/checkpoint_best.pt

sacreBLEU: Downloading http://statmt.org/wmt08/test.tgz to /root/.sacrebleu/wmt08/europarl/raw/wmt08_europarl.test.tgz
sacreBLEU: Extracting /root/.sacrebleu/wmt08/europarl/raw/wmt08_europarl.test.tgz to /root/.sacrebleu/wmt08/europarl/raw
2023-01-11 11:06:40 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-11 11:06:42 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': Fals